In [1]:
%run Reddit_import_and_functions.ipynb

In [90]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib

pd.set_option('display.max_colwidth',350)

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz, DecisionTreeClassifier
import pydotplus

from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets
from IPython.display import display, Markdown, Latex, Image

In [11]:
headers = pd.read_csv('masterFeaturesVector.csv')

In [64]:
with h5py.File("masterdata.hdf5",'a') as f :
    dset = f['Train1Ytrain']
    y_train = dset[:]
    print(dset.shape)
    print(y_train[220073:330109].shape)

(440145,)
(110036,)


In [5]:
with h5py.File("masterdata.hdf5",'a') as f :
    dset = f['columnsums/columnindextodrop']
    columnindextodrop = dset[:]

In [6]:
keepcols = set(range(17847)).symmetric_difference(columnindextodrop)
print(len(keepcols))
keepcols = list(keepcols)

10584


In [7]:
# Random Forest model rf1 trained on 0:220073 half of original train dset due to memory limitations
# Then tested on train1dset[220073:330109]
with h5py.File("masterdata.hdf5",'r') as f :
    dset = f['Train1dset']    
    print(dset.shape)
    df = pd.DataFrame(dset[220073:330109,keepcols], dtype='int16')
    print(df.shape)

(440145, 17847)
(110036, 10584)


In [14]:
# Memory problems just reading in from csv
# X_train = pd.read_csv('Train1Manageable.csv', dtype='int16', header=keepcols,
#                      compression='gzip', encoding='utf-8', engine='python')
# X_train.shape

MemoryError: 

In [9]:
rf = RandomForestClassifier(criterion='gini', verbose=1, n_jobs = 2)

In [12]:
rf_params = {
    'n_estimators': [10, 20, 30, 40],
    'max_depth': [100, 150, 200, 250], # after: None
    'min_samples_split': [10,14,18],
    'max_features' :['auto']
}
gs = GridSearchCV(rf, param_grid=rf_params, cv=3, verbose=1)
gs.fit(df, y_train[0:220073])

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:  2.4min finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.1s finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.5s finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:  2.5min finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.9s finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.8s finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:  2.4min finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.8s finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.5s finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:  5.1min finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.8s finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    4.4s finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:  5.1min finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.8s finished
[Parallel(n_jobs=2)]

[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    2.0s finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    4.3s finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:  6.0min finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    2.0s finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    4.5s finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:  6.0min finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    2.0s finished
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    5.2s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:  8.6min finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    2.8s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    5.0s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:  8.7min finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    2.9s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    6.7s finished
[Parallel(n_jobs=2)]

[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    7.2s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:  8.7min finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    2.4s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    6.5s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:  9.0min finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    2.9s finished
[Parallel(n_jobs=2)]: Done  30 out of  30 | elapsed:    5.9s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 11.8min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    3.7s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    8.3s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 11.8min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    3.7s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    8.9s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 11.9min finished
[Parallel(n_jobs=2)]

[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 11.4min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    3.7s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    8.7s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 13.0min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    4.2s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    7.5s finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:  3.4min finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.9s finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.9s finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:  3.7min finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.0s finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    2.6s finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:  3.6min finished
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    1.4s finished
[Parallel(n_jobs=2)]

0.9742449096436182


{'max_depth': 250,
 'max_features': 'auto',
 'min_samples_split': 14,
 'n_estimators': 40}

In [14]:
cross_val_score(gs.best_estimator_, df, y_train[0:220073])

[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 13.7min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    4.3s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 13.5min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    4.5s finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 13.3min finished
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    4.3s finished


array([0.97381335, 0.97334987, 0.97486266])

In [21]:
# joblib.dump(gs.best_estimator_, 'randomforestbest1.joblib')

['randomforestbest1.joblib']

In [22]:
gs.best_params_

{'max_depth': 250,
 'max_features': 'auto',
 'min_samples_split': 14,
 'n_estimators': 40}

In [9]:
rf1 = joblib.load('randomforestbest1.joblib')
rf1.feature_importances_

array([2.24699468e-06, 0.00000000e+00, 1.34087384e-05, ...,
       6.85501480e-06, 8.43048866e-08, 4.39969736e-06])

In [17]:
headers.loc[[0,8,10]]

,Unnamed: 0,features
0,0,aaa
8,8,aautomoderator
10,10,abandon


In [28]:
predictive_features = headers.loc[keepcols]
predictive_features['rf1_importances'] = rf1.feature_importances_

In [29]:
predictive_features.sort_values('rf1_importances', ascending=False)

,Unnamed: 0,features,rf1_importances
13275,13275,remove,0.041954
2923,2923,climate,0.034482
11379,11379,org,0.033271
3877,3877,crypto,0.029893
1610,1610,bitcoin,0.026848
5628,5628,energy,0.017483
17278,17278,water,0.016972
2658,2658,change,0.016854
1789,1789,blockchain,0.016370
3922,3922,cryptocurrency,0.016093


In [36]:
predictive_features[predictive_features['rf1_importances'] < .0001]['rf1_importances'].value_counts(bins=20)

(-0.0010001, 5e-06]    5723
(5e-06, 1e-05]         1671
(1e-05, 1.5e-05]        804
(1.5e-05, 2e-05]        463
(2e-05, 2.5e-05]        290
(2.5e-05, 3e-05]        228
(3e-05, 3.5e-05]        143
(3.5e-05, 4e-05]        127
(4e-05, 4.5e-05]         95
(4.5e-05, 5e-05]         75
(5e-05, 5.5e-05]         56
(5.5e-05, 6e-05]         51
(6e-05, 6.5e-05]         46
(7e-05, 7.5e-05]         38
(6.5e-05, 7e-05]         35
(7.5e-05, 8e-05]         29
(9.5e-05, 0.0001]        25
(8.5e-05, 9e-05]         24
(9e-05, 9.5e-05]         23
(8e-05, 8.5e-05]         21
Name: rf1_importances, dtype: int64

In [37]:
predictive_features[predictive_features['rf1_importances'] >= .0001]['rf1_importances'].value_counts(bins=20)

(-0.0009413000000000001, 0.00219]    525
(0.00219, 0.00429]                    46
(0.00429, 0.00638]                    16
(0.00638, 0.00847]                    10
(0.00847, 0.0106]                      4
(0.0168, 0.0189]                       3
(0.0106, 0.0127]                       3
(0.0127, 0.0147]                       3
(0.0147, 0.0168]                       2
(0.0399, 0.042]                        1
(0.0252, 0.0273]                       1
(0.0294, 0.0315]                       1
(0.0315, 0.0336]                       1
(0.0336, 0.0357]                       1
(0.0378, 0.0399]                       0
(0.021, 0.0231]                        0
(0.0231, 0.0252]                       0
(0.0273, 0.0294]                       0
(0.0357, 0.0378]                       0
(0.0189, 0.021]                        0
Name: rf1_importances, dtype: int64

In [10]:
X_test1 = df
#X_test1 = Train1[220073:330109]
# X_test2 = Train1[330109:440145]
rf1.score(X_test1, y_train[220073:330109])

[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    8.8s finished


0.9746537496819223

In [ ]:
pred1 = rf1.predict(X_test1)

In [45]:
pred_minus_actual = pd.read_csv('Train1allinfo.csv')

In [91]:
pred_minus_actual = pred_minus_actual.loc[220073:440145]
pred_minus_actual.loc[220073:(330109-1),'rf1_220073_330108'] = pred1 - y_train[220073:330109]
pred_minus_actual.loc[220073:(330109-1),['corpus','subreddit','rf1_220073_330108']].sort_values('rf1_220073_330108',ascending=False)

,corpus,subreddit,rf1_220073_330108
224613,narada collaborates electrify world second large energy storage capacity gross revenue excess oracletimes china narada collaborates electrify elec smart electricity solution,0,1.0
277278,note fee getmonero org note fee,0,1.0
288006,just release stellar lumen rocket fuel hype encumber dilution lack transparency medium lightcrypto stellar lumen rocket fuel hype encumber dilution lack transparency,0,1.0
281296,big head lose theirs blame trust men doubt make allowance doubt wait tire wait lie don deal lie hat don way hat don look good talk wise dream make dream master think make thought aim meet triumph disaster treat impostor just bear hear truth spoken twist knave make trap fool watch thing give life broken stoop build worn tool make heap winning r...,0,1.0
310670,futuresplatform blog game nation state futuresplatform blog game nation state,0,1.0
329351,government offering bounty pump dump group networkassociates gov offering bounty pump dump group,0,1.0
252399,paid anti human progress officer anti detection field sector refer milky way time come clean remove xlznx paid antihuman progress officer,0,1.0
291643,swipecrypto data privacy landscape change forever swipecrypto swipecrypto swipecrypto data privacy landscape,0,1.0
236341,coffee time check news gfycat gifs wetimprobableanaconda,0,1.0
283333,treasury department don muh bitcoins home treasury gov site default file financial creates economic opportunity nonbank financi pdf,0,1.0


In [92]:
confusion_matrix(y_train[220073:330109], pred1) / 110036

array([[0.66040205, 0.00721582],
       [0.01813043, 0.3142517 ]])

In [94]:
pred_minus_actual.to_csv('pred_minus_actual.csv')